This tests the accuracy of a differential operator on a polynomial of the given order. If the operator is working to said order, then the `Actual` and `Theoretical` outputs should match (except possibly at the edges.)

In [1]:
import os.path
from scipy import *
import numpy as np
from numpy import *
from numpy import linalg as LA
from scipy import linalg as LA2
import sympy as sympy
import sys as sys
import time
import matplotlib.pyplot as plt
import itertools as it
from IPython.core.display import HTML

sys.path.append('/Users/sashacurcic/SashasDirectory/ANAG/FV_MG/')
from Modules import BasicTools as BT
from Modules import WaveTools as WT
from Modules import PlotTools as PT
from Modules import FFTTools as FFTT
from Modules import OperatorTools as OT
from Modules import GridTransferTools as GTT
from Modules import TestTools as TT
from Modules import SolverTools as ST

display(HTML("<style>pre { white-space: pre !important; }</style>"))
np.set_printoptions( linewidth = 10000, threshold = 100000)

This cell takes all of the input parameters.

In [2]:
# Grid parameters.
nh_min = 16
refRatio = 2
cfInter = int(nh_min / refRatio)

# Time integration parameters.
deriv = 'CD'
# RK = 4
order = 6

This cell contains the relevant switch information.

In [3]:
# TimeIntFunc, DiffFunc = TT.SolverSwitch(deriv, RK)

This cell instantiates the Grid and the Physics objects.

In [4]:
omega = BT.Grid(nh_min)
finehalf = list(np.arange(int(nh_min / refRatio)))# + int(nh_min / refRatio))
omega.AddPatch(refRatio, finehalf)
degFreed = omega.degFreed

This cell runs a test on the derivative operator on a cell-averaged polynomial which matches the given order.  
$P (x) = \sum_{k = 0}^{o} x^{k}$  
$<P (x)> = \frac{1}{\Delta x} \int_{x}^{x + \Delta x} P (x')$d$x'$  
$<P (x)> = \frac{1}{\Delta x} \sum_{k = 1}^{o} \frac{1}{k} [(x + \Delta x)^{k} - x^{k}]$  
$<\frac{d P (x)}{d x}> = \sum_{k = 1}^{o} [(x + \Delta x)^{k} - x^{k}]$  

In [5]:
for i in range(order + 1, degFreed - order - 1):
    print('h:', i)
    TT.DerivPolyTest(omega, deriv, order, deriv = 0, matInd = i, printOut = False)
    print('')

h: 7
centCellInd is 8
polyInterp is [  84.         -377.99999999  755.99999999 -839.99999999  539.99999999 -189.           28.        ]
centCellInd is 9
polyInterp is [  28. -112.  210. -224.  140.  -48.    7.]
centCellInd is 10
polyInterp is [  7. -21.  35. -35.  21.  -7.   1.]
centCellInd is 0
polyInterp is [  84. -378.  756. -840.  540. -189.   28.]
centCellInd is 1
polyInterp is [  28. -112.  210. -224.  140.  -48.    7.]
centCellInd is 2
polyInterp is [  7. -21.  35. -35.  21.  -7.   1.]
centCellInd is 8
polyInterp is [  28. -112.  210. -224.  140.  -48.    7.]
centCellInd is 9
polyInterp is [  7. -21.  35. -35.  21.  -7.   1.]
centCellInd is 0
polyInterp is [  28. -112.  210. -224.  140.  -48.    7.]
centCellInd is 1
polyInterp is [  7. -21.  35. -35.  21.  -7.   1.]
centCellInd is 8
polyInterp is [  7. -21.  35. -35.  21.  -7.   1.]
centCellInd is 0
polyInterp is [  7. -21.  35. -35.  21.  -7.   1.]
centCellInd is 7
polyInterp is [   7.  -48.  140. -224.  210. -112.   28.]
centC

polyInterp is [   84.          -559.99999999  1574.99999997 -2399.99999997  2099.99999998 -1007.99999999   210.        ]
centCellInd is 14
polyInterp is [ 0.04101562 -0.23171165  1.0131392   0.24360795 -0.08451705  0.0209517  -0.0024858 ]
centCellInd is 15
polyInterp is [ 0.02734375 -0.1241714   0.2985322   0.9313447  -0.1624053   0.0329072  -0.00355114]
centCellInd is 21
polyInterp is [  1.  -7.  21. -35.  35. -21.   7.]
centCellInd is 22
polyInterp is [   7.          -48.00000001  140.00000002 -224.00000002  210.00000002 -112.00000001   28.        ]
centCellInd is 23
polyInterp is [  28.         -189.00000003  540.00000007 -840.00000009  756.00000007 -378.00000003   84.        ]
[[  -78.4          192.          -240.           213.33333333  -120.            38.4           -5.33333333     0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.             0.  

In [6]:
coefs = 1. / (np.arange(order + 1) + 1)
coefs = np.append(1, coefs)[::-1]
P = np.poly1d(coefs) - 1
print(coefs)
print(P)
print('')

[0.14285714 0.16666667 0.2        0.25       0.33333333 0.5        1.         1.        ]
        7          6       5        4          3       2
0.1429 x + 0.1667 x + 0.2 x + 0.25 x + 0.3333 x + 0.5 x + 1 x



In [7]:
print(order, degFreed - order - 2)

6 16


In [8]:
A = np.arange(16).reshape(4, 4)
print(A)
print('')
print(sum(A, axis = 1))

[[ 0  1  2  3]
 [ 4  5  6  7]
 [ 8  9 10 11]
 [12 13 14 15]]

[ 6 22 38 54]


In [9]:
A = [28., -189.00000003, 540.00000007, -840.00000009, 756.00000007, -378.00000003, 84.]
B = [2.66666666, -19.2, 60.00000001, -106.66666665, 120., -96., 39.2]
C = [A[i] / B[i] for i in range(7)]
print(C)
print('')


[10.50000002625, 9.8437500015625, 8.999999999666667, 7.875000002074219, 6.300000000583334, 3.9375000003125002, 2.142857142857143]

